## ref : https://nlp.seas.harvard.edu/2018/04/03/attention.html

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import math, copy, time
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context='talk')

# torch : 1.9.0+cu102  |  np : 1.19.5  |  seaborn : 0.11.1
print(f'torch : {torch.__version__}  |  np : {np.__version__}  |  seaborn : {seaborn.__version__}')

torch : 1.9.0+cu102  |  np : 1.19.5  |  seaborn : 0.11.1
